In [57]:
import requests as r
import bs4
import urllib
from newspaper import Article


In [ ]:
import nltk
nltk.download('punkt')

In [61]:
url_main = "https://www.google.com/search"
query = "RCB"
tbm = "nws"
start = "15"

URL = f"{url_main}?q={query}&tbm={tbm}&start={start}"


In [62]:

hit = r.get(url=URL)
html = hit.content

link_list = []
cleaned_urls = []

soup = bs4.BeautifulSoup(html, 'html.parser')
tags  = soup.find_all('a')

for link in tags:
    link_list.append(link.get('href')) 


for i in link_list:
    if i.startswith('/url?q=https://www'):
        link = i.replace('/url?q=', '')
        cleaned_urls.append(link)



In [69]:
cleaned_urls
working_urls = []
for i in cleaned_urls:
    req = r.get(i)
    
    if req.status_code == 200:
        working_urls.append(i)
        

In [72]:
for i in working_urls:
    news_article = Article(i)
    news_article.download() """request.get(url) == response.content"""
    news_article.parse() """  using python string methods"""  
    # news_article.nlp() """spacy""" """"
    # news_article.text """ text from the article"""

    print(f"Title: {news_article.title}")
    print(f"authors: {news_article.authors}")
    print(f"keywords: {news_article.keywords}")
    print(f"url: {news_article.url}")
    print(f"text: {news_article.text}")

Title: Sports News: Cricket News, Football News, Hockey News, Sports Breaking News
authors: ['Business Standard']
keywords: []
url: https://www.business-standard.com/article/sports/ipl-2022-rcb-vs-csk-live-updates-dhoni-virat-ipl-points-table-playoffs-122050400410_1.html&sa=U&ved=2ahUKEwiXjNHfyoL4AhUDT2wGHcZNBak4DxDF9AF6BAgFEAI&usg=AOvVaw2LI4vh98WgKFYET9wGDOOJ
text: A rejuvenated Indian men's hockey team would like to put up a much-improved performance when it takes on Japan in its first ...
Title: TATA IPL 2022, Match 49: RCB Vs CSK – Match Report
authors: []
keywords: []
url: https://www.iplt20.com/news/3802/tata-ipl-2022-match-49-rcb-vs-csk-match-report&sa=U&ved=2ahUKEwiXjNHfyoL4AhUDT2wGHcZNBak4DxDF9AF6BAgHEAI&usg=AOvVaw3BeTwW-nYVUfQdN7a04MeN
text: TATA IPL 2022, Match 49: RCB Vs CSK – Match Report

The Royal Challengers Bangalore (RCB) were back to winning ways in the TATA Indian Premier League (IPL) 2022 after registering a victory against Chennai Super Kings (CSK) by 13 runs on W

In [68]:
# use urllib and find the count of how many ruls have the same domain in cleaned_urls
# look_up = {}
# for i in cleaned_urls:
#     url = urllib.parse.urlparse(i)
#     # print(url)
#     domain = url.netloc
#     if domain in look_up:
#         look_up[domain] += 1
#     else:
#         look_up[domain] = 1
#     # print(domain)

# print(look_up)

for i in cleaned_urls:
    req = r.get(i)
    print(req.headers)
    print(req.status_code)

{'Content-Encoding': 'gzip', 'Content-Type': 'text/html; charset=utf-8', 'Pragma': 'no-cache', 'Server': 'Apache', 'Strict-Transport-Security': 'max-age=31536000; env=HTTPS', 'X-Mod-Pagespeed': '1.13.35.1-0', 'Content-Length': '32053', 'Cache-Control': 'no-cache, no-store, must-revalidate, post-check=0, pre-check=0', 'Date': 'Sat, 28 May 2022 16:16:21 GMT', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding'}
200
{'Content-Type': 'text/html', 'Content-Length': '51', 'Date': 'Sat, 28 May 2022 16:16:21 GMT', 'Connection': 'keep-alive', 'Set-Cookie': 'ht-location=IN; path=/', 'cachettl': '', 'X-Frame-Options': 'SAMEORIGIN'}
401
{'Date': 'Sat, 28 May 2022 16:16:22 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Set-Cookie': 'AWSALB=y/1C+Fys3xAiWK2nez3UoKutvFd+yNRTW4/2uLqsYL2hNog6SBv7UfICcBvWYAVVdgLE9bW15bvBhaaWK9+KqLaOIyijy7OpgH1cyta87rHxQjP9KMYUxiMcJ776; Expires=Sat, 04 Jun 2022 16:16:22 GMT; Path=/, AWSALBCORS=y/1C+Fys3xAiWK2nez3UoKutvFd+yNRTW4/2uLqsYL2hNog6SBv7UfICcBvWYAVVdgLE9bW15

In [ ]:
# url = 'https://eand.co/the-age-of-extinction-is-here-some-of-us-just-dont-know-it-yet-7001f5e0c79a'
# url = "https://www.hindustantimes.com/cities/pune-news/covid19-maharashtra-reports-first-cases-of-ba-4-ba-5-subvariants-of-omicron-101653741892079.html"
url = "https://www.financialexpress.com/express-mobility/ford-joins-first-movers-coalition-moves-a-step-ahead-to-carbon-neutrality/2540483/"
article = Article(url)
article.download()
article.parse()
article.nlp(

)
article.publish_date

In [ ]:
article.authors

In [ ]:
print(article.keywords)
article.top_image


In [ ]:
article.keywords

In [73]:
!pip install spacy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 296.0 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 KB 364.4 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 282.4 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.1/128.1 KB 590.8 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.6/457.6 KB 365.8 kB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 KB 390.3 kB/s eta 0:00:001m553.1 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 668.8/668.8 KB 235.6 kB/s eta 0:00:001m232.2 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 63.5 kB/s eta 0:00:00m eta 0:00:010:00:06
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 KB 172.6 kB/s eta 0:00:00 kB/s eta 0:00:01
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.9.0
    Uni

In [74]:
import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.